<img src='https://github.com/Ikomia-dev/notebooks/blob/main/examples/img/banner_ikomia.png?raw=true'>




# Easy semantic segmentation with MMSegmentation

**MMSegmentation** is part of the OpenMMLab project and is developed by the Multimedia Laboratory at the Chinese University of Hong Kong. It specializes in semantic segmentation, a vital component in the field of computer vision. 

It offers an extensive collection of segmentation models and algorithms, making it a go-to choice for both researchers and practitioners in the field.



![MMSegmentation illustration](https://github.com/open-mmlab/mmsegmentation/blob/main/resources/seg_demo.gif?raw=true)

## Setup

You need to install Ikomia Python API with pip


In [ ]:
!pip install ikomia

## Run MMSegmentation on your image

---

**-Google Colab ONLY- Restart runtime after the first run of the workflow below** 

Click on the "RESTART RUNTIME" button at the end the previous window.

---

In [ ]:
from ikomia.dataprocess.workflow import Workflow

# Init your workflow
wf = Workflow()

# Add algorithm
segmentor = wf.add_task(name="infer_mmlab_segmentation", auto_connect=True)

segmentor.set_parameters({
        "model_name": "pspnet",
        "model_config": "pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py",
        "cuda": "True",
    })


# Run the workflow on image
wf.run_on(url="https://github.com/open-mmlab/mmsegmentation/blob/main/demo/demo.png?raw=true")


In [ ]:
from ikomia.core import IODataType
from ikomia.utils.displayIO import display

from PIL import ImageShow
ImageShow.register(ImageShow.IPythonViewer(), 0)

# Display the results
display(segmentor.get_image_with_mask())

In [ ]:
# Display legend
results = segmentor.get_results()
display(results.get_legend())

## List of parameters

- **model_name** (str, default="maskformer"): model name. 
- **model_config** (str, default="maskformer_r50-d32_8xb2-160k_ade20k-512x512"): name of the model configuration file.
- **config_file** (str, default=""): path to model config file (only if *use_custom_model=True*). The file is generated at the end of a custom training. Use algorithm ***train_mmlab_detection*** from Ikomia HUB to train custom model.
- **model_weight_file** (str, default=""): path to model weights file (.pt) (only if *use_custom_model=True*). The file is generated at the end of a custom training.
- **cuda** (bool, default=True): CUDA acceleration if True, run on CPU otherwise.

MMLab framework for object detection and instance segmentation offers a large range of models. To ease the choice of couple (model_name/model_config), you can call the function *get_model_zoo()* to get a list of possible values.


In [ ]:
from ikomia.dataprocess.workflow import Workflow

# Init your workflow
wf = Workflow()

# Add algorithm
segmentor = wf.add_task(name="infer_mmlab_segmentation", auto_connect=True)

# Get list of possible models (model_name, model_config)
models = segmentor.get_model_zoo()

for model in models:
    print(model)

### Run MMSegmentation on Video
*Note: The video stream will work on local only, not on Google Colab*

In [ ]:
from ikomia.dataprocess.workflow import Workflow
from ikomia.utils.displayIO import display
import cv2

# Use a path to your video
video_input = 'PATH/TO/YOUR/VIDEO.mp4'  # Example video from Pexels: https://www.pexels.com/video/busy-street-in-new-york-854614/
output_path = 'video_output.mp4'

# Init your workflow
wf = Workflow()

# Add algorithm
segmentor = wf.add_task(name="infer_mmlab_segmentation", auto_connect=True)

segmentor.set_parameters({
    "model_name": "segformer",
    "model_config": "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024",
    "cuda": "True",
})

# Open the video file from URL
stream = cv2.VideoCapture(video_input)
if not stream.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties for the output
frame_width = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = stream.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

while True:
    # Read image from stream
    ret, frame = stream.read()
    if not ret:
        print("Info: End of video or error.")
        break

    # Run the workflow on current frame
    wf.run_on(array=frame)

    # Get results
    image_out = segmentor.get_output(0)
    obj_detect_out = segmentor.get_output(1)

    # Convert the result to BGR color space for displaying
    img_out = image_out.get_image_with_mask_and_graphics(obj_detect_out)
    img_res = cv2.cvtColor(img_out, cv2.COLOR_RGB2BGR)

    # Save the resulting frame
    out.write(img_res)  # This should be img_res instead of img_out if you intend to save the converted BGR image

    # Display
    display(img_res, title="MMSeg Semantic Segmentation", viewer="opencv")

    # Press 'q' to quit the video processing
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything after the loop
stream.release()
out.release()
cv2.destroyAllWindows()